In [ ]:
# ======================================
# 03_backtest_and_analysis.ipynb
# Corre backtest multi-asset con los modelos ya entrenados
# y visualiza equity, trades y métricas clave.
# ======================================

from src.utils import load_config, ensure_dirs
from src.data_loader import load_universe_data
from src.features import build_features
from src.labeler import make_labels
from src.backtester_multi import run_full_backtest

import pandas as pd
import matplotlib.pyplot as plt
import pprint
from IPython.display import display

# --- 1. setup inicial ---
cfg = load_config()
ensure_dirs()

print("=== CONFIG ESTRATEGIA ===")
pprint.pprint(cfg["strategy"])

with open("data/tickers.txt") as f:
    tickers = [line.strip() for line in f if line.strip()]

print("Tickers cargados:", tickers)

# --- 2. reconstrucción de data_map (igual que pipeline) ---
raw_map = load_universe_data(
    tickers,
    cfg["data"]["lookback_days"],
    cfg["data"]["interval"]
)

data_map = {}
for t, df in raw_map.items():
    feat_df = build_features(df)
    labeled_df = make_labels(
        feat_df,
        cfg["strategy"]["horizon"],
        cfg["strategy"]["tp"],
        cfg["strategy"]["sl"]
    )
    data_map[t] = labeled_df

# --- 3. correr backtest multi-asset ---
feature_cols = cfg["features"]["include"]

equity_result, summary = run_full_backtest(
    data_map=data_map,
    feature_cols=feature_cols,
    model_dir="models",
    horizon=cfg["strategy"]["horizon"],
    tp=cfg["strategy"]["tp"],
    sl=cfg["strategy"]["sl"],
    base_prob_threshold=cfg["strategy"]["base_prob_threshold"],
    initial_capital=cfg["strategy"]["capital"],
    risk_fraction=cfg["strategy"]["risk_fraction"]
)

print("=== RESUMEN GLOBAL ===")
pprint.pprint(summary)

equity_df = equity_result["equity_curve"]
trades_df = equity_result["trades_enriched"]

# --- 4. curva de equity ---
plt.figure(figsize=(10,5))
plt.plot(equity_df["time"], equity_df["capital"], label="Equity Curve")
plt.title("📈 Curva de Capital Simulada (Multi-Asset)")
plt.xlabel("Tiempo")
plt.ylabel("Capital (USD)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# --- 5. ranking por ticker ---
print("\n=== 🏆 Ranking por Ticker (PnL USD total) ===")
for t, val in summary["by_ticker"].items():
    print(f"{t:6s} -> {val:10.2f} USD")

# --- 6. mejores y peores trades ---
print("\n=== 🔝 Top 5 Trades Ganadores ===")
display(trades_df.sort_values("pnl_usd", ascending=False).head())

print("\n=== 💀 Top 5 Trades Perdedores ===")
display(trades_df.sort_values("pnl_usd", ascending=True).head())

# --- 7. distribución de retornos ---
plt.figure(figsize=(8,4))
plt.hist(trades_df["pnl_pct"]*100, bins=50, edgecolor="gray")
plt.title("Distribución de Retornos por Trade (%)")
plt.xlabel("Retorno (%)")
plt.ylabel("Frecuencia")
plt.grid(alpha=0.3)
plt.show()

print("\nListo. Estos resultados alimentan el dashboard (próxima etapa).")
